In [1]:
'''
The below is a script for cleaning the housing market data after aggregation. Please see comments below 
on specific action taken. 
'''

import pandas as pd

# File path, loading the data, basic info and unique values
data_file = 'C:/Users/ryanm/Desktop/Projects/3/Datasets/full_housing_data.csv'
zip_city_file = 'C:/Users/ryanm/Desktop/Projects/3/Datasets/zip_city_check.csv'

df = pd.read_csv(data_file)

'''
# Inspect data
df.info()
df.nunique()
'''

# Remove duplicates
df = df.drop_duplicates()

# Removing "0 non-null" (MLS ID, Number of days on Zillow) and redundant values
columns_to_drop = ['MLS ID',
                   'Number of days on Zillow', 
                   'Living area unit', 
                   'Is owner occupied', 
                   'Property owned by Zillow'   
]

df.drop(columns=columns_to_drop, inplace=True)

# Rename column, as all units are sqft
df.rename(columns={'Living area': 'Living area - square footage'}, inplace=True)

# Convert 'Lot/land area' to acreage, drop unit column, and rename the column
sqft_to_acres = 1 / 43560
df['Lot/land area'] = df.apply(lambda row: row['Lot/land area'] * sqft_to_acres if row['Lot/land area unit'] == 'sqft' else row['Lot/land area'], axis=1)
df.drop(columns=['Lot/land area unit'], inplace=True)
df.rename(columns={'Lot/land area': 'Lot/land area (acreage)'}, inplace=True)

# Filter rows where State is 'PA', and drop 'State' and 'Country' columns
df = df[df['State'] == 'PA']
df.drop(columns=['State', 'Country'], inplace=True)

# Convert 'Sold date (MM/DD/YYYY)' to DateTime
df['Sold date (MM/DD/YYYY)'] = pd.to_datetime(df['Sold date (MM/DD/YYYY)'], format='%m/%d/%Y')

# Assign columns to appropriate data types fill missing values
object_nulls = ['Property URL', 
                'Broker name',
                'Broker agent', 
                'Property unit number', 
                'Street address', 'City',
                'Listing Type'
]

number_nulls = ['Property price (USD)', 
                'Zestimate (USD)', 
                'Rent zestimate (USD per month)', 
                'Tax assessed value (USD)', 
                'Living area - square footage', 
                'Price per living area unit (USD)', 
                'Lot/land area (acreage)', 
                'Price per lot/land area unit (USD)', 
                'Bedrooms', 
                'Bathrooms', 
                'Zip', 
                'Latitude', 
                'Longitude'
]

df[object_nulls] = df[object_nulls].fillna('NULL')
df[number_nulls] = df[number_nulls].fillna(0)
df['Sold date (MM/DD/YYYY)'] = df['Sold date (MM/DD/YYYY)'].fillna(pd.to_datetime('1900-01-01', format='%Y-%m-%d'))

# Most of the columns ought to be ints rather than floats. Below is the conversion.
float_conversions = ['Zestimate (USD)', 
                     'Rent zestimate (USD per month)', 
                     'Tax assessed value (USD)', 
                     'Living area - square footage', 
                     'Bedrooms', 
                     'Bathrooms', 
                     'Zip'
]

df[float_conversions] = df[float_conversions].astype('int64')

'''
Here I noticed an issue with the "City" column where the names of cities weren't standardized. For example, 
East Pittsburgh and E Pittsburgh should both be under East Pittsburgh, some values show Bradford woods while 
others show Bradfordwoods, etc. 

I built a web scraper to standardize city names, pulling the zip code from each row and populating the 
recognized postal address, then used the resulting df to cross reference the proper city names.

You can find the seperate script for the scraper on my GitHub. Below is further cleaning with the df from the scraper.
'''

# Load city_check_df
city_check_df = pd.read_csv(zip_city_file)

# Merge dataframes and update 'City' column
merged_df = df.merge(city_check_df, left_on='Zip', right_on='Zip Code', how='left')
merged_df['City'] = merged_df['City Name']
merged_df.drop(['Zip Code', 'City Name'], axis=1, inplace=True)
df = merged_df

# Drop rows with missing 'City', as they're not showing Allegheny County
df.dropna(subset=['City'], inplace=True)

'''
I also noticed both the "bedroom" and "bathroom" column both has 1 obscure value each, showing far
too many what there should be for the listing upon inspection. 
'''

# Find and remove the rows with the exorbitant values in the 'Bedrooms' and 'Bathrooms' columns
max_bedrooms_index = df['Bedrooms'].idxmax()
max_bathrooms_index = df['Bathrooms'].idxmax()
rows_to_remove = {max_bedrooms_index, max_bathrooms_index}
df = df.drop(rows_to_remove)

# Save the cleaned dataset
df.to_csv('C:/Users/ryanm/Desktop/Projects/3/Datasets/cleaned_housing_data.csv', index=False)

print("Cleaned")


Cleaned
